# Imports

In [1]:
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme.expressions import Beta, Variable, exp, Expression
from biogeme.models import loglogit
from biogeme.tools import likelihood_ratio_test
from biogeme.results import compile_estimation_results
from biogeme.models import loglogit,  boxcox
from biogeme.models.piecewise import piecewise_formula
from biogeme.models import lognested
from biogeme.nests import OneNestForNestedLogit, NestsForNestedLogit
from biogeme.biogeme import BIOGEME

import pickle

import numpy as np
import os

from scipy.stats import chi2

# Data & Variables

In [2]:
# Define the relative path to the data folder
file_path = os.path.join(os.pardir, 'lpmc01.dat')

#file_path = os.path.join(data_folder, 'lpmc01.dat')

df = pd.read_csv(file_path, sep = '\t')
df['age_normalized'] = (df['age'] - df['age'].mean()) / df['age'].std()
df['age_scaled'] = df['age'] / df['age'].max()
df['cost_driving'] = df['cost_driving_ccharge'] + df['cost_driving_fuel']
df['dur_pt'] = df['dur_pt_access'] + df['dur_pt_rail'] + df['dur_pt_int'] + df['dur_pt_bus']

# max_cost = df['cost_driving'].max() if df['cost_driving'].max() > df['cost_transit'].max() else df['cost_transit'].max()
# df['cost_driving'] = df['cost_driving']/max_cost
# df['cost_transit'] = df['cost_transit']/max_cost


database1 = db.Database('lpmc01', df)


# Define the given veriables 
dur_pt = Variable('dur_pt')
cost_driving = Variable('cost_driving')
age_scaled = Variable('age_scaled')
trip_id = Variable('trip_id')
household_id = Variable('household_id')
person_n = Variable('person_n')
trip_n = Variable('trip_n')
travel_mode = Variable('travel_mode')
purpose = Variable('purpose')
fueltype = Variable('fueltype')
faretype = Variable('faretype')
bus_scale = Variable('bus_scale')
survey_year = Variable('survey_year')
travel_year = Variable('travel_year')
travel_month = Variable('travel_month')
travel_date = Variable('travel_date')
day_of_week = Variable('day_of_week')
start_time = Variable('start_time')
age = Variable('age')
female = Variable('female')
driving_license = Variable('driving_license')
car_ownership = Variable('car_ownership')
distance = Variable('distance')
dur_walking = Variable('dur_walking')
dur_cycling = Variable('dur_cycling')
dur_pt_access = Variable('dur_pt_access') # Predicted total access and egress time for public transport route in hours
dur_pt_rail = Variable('dur_pt_rail')
dur_pt_bus = Variable('dur_pt_bus')
dur_pt_int = Variable('dur_pt_int') # Time taken (hrs) at each interchange point
pt_interchanges = Variable('pt_interchanges')   # Number of interchange points in public transport route
dur_driving = Variable('dur_driving')
cost_transit = Variable('cost_transit')
cost_driving_fuel = Variable('cost_driving_fuel')   # Estimated fuel cost of driving route in GBP
cost_driving_ccharge = Variable('cost_driving_ccharge')  # Estimated congestion charge cost of driving route in GBP
driving_traffic_percent = Variable('driving_traffic_percent')



# Define transport availability
# Assume pt, walking, cycle always available, with car availability depending on number of cars per household. From the data, 
# people without driving licenses choose driving as their mode of transport (eg. row 28). 
av_drive =  (car_ownership > 0)
av_pt =1
av_walk = 1
av_cycle = 1

variable_names = ['dur_pt', 'cost_driving', 'age_scaled']  # Replace with your variable name
for variable_name in variable_names:
    if variable_name in database1.data.columns:
        print(f"'{variable_name}' exists in the database.")
    else:
        print(f"'{variable_name}' does NOT exist in the database.")



# Define pt_cost (not needed)
# Original paper, page 31: "Public transport fares are determined for single trips using Oystercard/contactless payment."
# Therefore, cost_transit should already consider faretype and bus_scale

database = db.Database('lpmc01', df)
variable_names = ['dur_pt', 'cost_driving', 'age_scaled']  # Replace with your variable name
for variable_name in variable_names:
    if variable_name in database1.data.columns:
        print(f"'{variable_name}' exists in the database.")
    else:
        print(f"'{variable_name}' does NOT exist in the database.")

# Define driving cost
cost_driving = cost_driving_ccharge + cost_driving_fuel

# Define time taken by each mode of transport
dur_pt = dur_pt_access + dur_pt_int + dur_pt_bus + dur_pt_rail  # Public transport (external) time 

'dur_pt' exists in the database.
'cost_driving' exists in the database.
'age_scaled' exists in the database.
'dur_pt' exists in the database.
'cost_driving' exists in the database.
'age_scaled' exists in the database.


# Model Definition

In [3]:
# Assume every mode of transport is available
availability_walk = 1  
availability_cycle = 1  
availability_pt = 1     
availability_drive = 1

availability = {
    1: availability_walk,   # Walking
    2: availability_cycle,  # Cycling
    3: availability_pt,     # Public Transport
    4: availability_drive   # Driving
}

# Define alternative-specific parameters for travel time
B_TIME_WALK = Beta('B_TIME_WALK', 0, None, None, 0)
B_TIME_CYCLE = Beta('B_TIME_CYCLE', 0, None, None, 0)
B_TIME_PT = Beta('B_TIME_PT', 0, None, None, 0)
B_TIME_DRIVE = Beta('B_TIME_DRIVE', 0, None, None, 0)

ASC_CYCLE = Beta('ASC_CYCLE', 0, None, None, 0)
ASC_PT = Beta('ASC_PT', 0, None, None, 0)
ASC_DRIVE = Beta('ASC_DRIVE', 0, None, None, 0)

# Define generic parameters for cost and travel time
B_COST = Beta('B_COST', 0, None, None, 0)

B_TIME_WALK_AGE = Beta('B_TIME_WALK_AGE', 0, None, None, 0)
B_TIME_CYCLE_AGE = Beta('B_TIME_CYCLE_AGE', 0, None, None, 0) 
B_TIME_PT_AGE = Beta('B_TIME_PT_AGE', 0, None, None, 0)
B_TIME_DRIVE_AGE = Beta('B_TIME_DRIVE_AGE', 0, None, None, 0) * age_scaled

## BoxCox

In [4]:
LAMDA_COST = Beta('lambda_cost', 1, -10, 10, 0)
bx_cost_pt = boxcox(cost_transit, LAMDA_COST)
bx_cost_driving = boxcox(cost_driving, LAMDA_COST)

# Utility functions with interaction terms
# Updated utility functions with age interaction for travel time
V_WALK = (B_TIME_WALK + B_TIME_WALK_AGE * age_scaled) * dur_walking
V_CYCLE = ASC_CYCLE + (B_TIME_CYCLE + B_TIME_CYCLE_AGE* age_scaled) * dur_cycling
V_PT = ASC_PT + B_COST * bx_cost_pt + (B_TIME_PT + B_TIME_PT_AGE* age_scaled) * dur_pt
V_DRIVE = ASC_DRIVE + B_COST * bx_cost_driving + (B_TIME_DRIVE + B_TIME_DRIVE_AGE * age_scaled) * dur_driving

# Associate utility functions with the mode choice
V = {
    1: V_WALK,    # Walking
    2: V_CYCLE,   # Cycling
    3: V_PT,      # Public Transport
    4: V_DRIVE    # Driving
}

# Specify the model
model_3_bx = loglogit({1: V_WALK, 2: V_CYCLE, 3: V_PT, 4: V_DRIVE}, availability, travel_mode)

### Results

In [5]:
biogeme_bx = bio.BIOGEME(database, model_3_bx)
biogeme_bx.modelName = 'model_3_bx'

results_m3_bx = biogeme_bx.estimate()

print("Estimation results for Model 3 Box Cox:")
print(results_m3_bx.get_estimated_parameters())

Estimation results for Model 3 Box Cox:
                     Value  Rob. Std err  Rob. t-test  Rob. p-value
ASC_CYCLE        -4.744093      0.216394   -21.923446  0.000000e+00
ASC_DRIVE        -2.858211      0.210642   -13.569064  0.000000e+00
ASC_PT           -2.787671      0.160238   -17.397042  0.000000e+00
B_COST           -0.455481      0.052597    -8.659809  0.000000e+00
B_TIME_CYCLE     -5.100008      0.771066    -6.614229  3.734923e-11
B_TIME_CYCLE_AGE -4.732287      1.626277    -2.909890  3.615556e-03
B_TIME_DRIVE     -5.887612      0.457107   -12.880167  0.000000e+00
B_TIME_DRIVE_AGE -2.231037      1.638828    -1.361361  1.733995e-01
B_TIME_PT        -2.588571      0.394738    -6.557687  5.464895e-11
B_TIME_PT_AGE    -3.030230      0.853149    -3.551820  3.825768e-04
B_TIME_WALK      -7.877341      0.506051   -15.566313  0.000000e+00
B_TIME_WALK_AGE  -4.154185      0.952283    -4.362341  1.286781e-05
lambda_cost       0.320712      0.084479     3.796356  1.468384e-04


In [6]:
print(results_m3_bx.print_general_statistics())

Number of estimated parameters:	13
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-6931.472
Final log likelihood:	-4194.685
Likelihood ratio test for the init. model:	5473.573
Rho-square for the init. model:	0.395
Rho-square-bar for the init. model:	0.393
Akaike Information Criterion:	8415.371
Bayesian Information Criterion:	8500.094
Final gradient norm:	3.7164E-02
Nbr of threads:	16



### Test against Model 2

In [7]:
folder_path = os.path.join(os.pardir, 'Model_2')
file_path = os.path.join(folder_path, 'Model_2B.pickle')

# open a file, where you stored the pickled data
file = open(file_path, 'rb')

# dump information to that file
results_m2 = pickle.load(file)

# close the file
file.close()

In [8]:
loglikehood_m3_bx = results_m3_bx.data.logLike
num_params_m3_bx = results_m3_bx.data.nparam

loglikehood_m2 = results_m2.logLike
num_params_m2 = results_m2.nparam

# Calculate the LR statistic
LR = 2 * (loglikehood_m3_bx - loglikehood_m2)

# Degrees of freedom
df = num_params_m3_bx - num_params_m2

# Critical value at 0.05 significance level
critical_value = chi2.ppf(0.95, df)

print("Likelihood Ratio:", LR)
print("Degrees of Freedom:", df)
print("Critical Chi-Square Value (0.05 significance):", critical_value)

if LR > critical_value:
    print("Model 3 Box Cox is significantly better than Model 2.")
else:
    print("No significant improvement in Model 3 Box Cox over Model 2.")

Likelihood Ratio: -1.8605945036906633
Degrees of Freedom: 1
Critical Chi-Square Value (0.05 significance): 3.841458820694124
No significant improvement in Model 3 Box Cox over Model 2.


## Power Series

In [9]:
square_tt_coef = Beta('square_cost_coef', 0, None, None, 0)
cube_tt_coef = Beta('cube_cost_coef', 0, None, None, 0)

def power_series(the_variable: Expression) -> Expression:
    """Generate the expression of a polynomial of degree 3

    :param the_variable: variable of the polynomial
    """
    return (
        the_variable
        + square_tt_coef * the_variable**2
        + cube_tt_coef * the_variable * the_variable**3
    )

In [10]:
power_cost_driving = power_series(cost_driving)
power_cost_pt = power_series(cost_transit)

# Utility functions with interaction terms
V_WALK = (B_TIME_WALK + B_TIME_WALK_AGE * age_scaled) * dur_walking
V_CYCLE = ASC_CYCLE + (B_TIME_CYCLE + B_TIME_CYCLE_AGE* age_scaled) * dur_cycling
V_PT = ASC_PT + B_COST * power_cost_pt + (B_TIME_PT + B_TIME_PT_AGE* age_scaled) * dur_pt
V_DRIVE = ASC_DRIVE + B_COST * power_cost_driving + (B_TIME_DRIVE + B_TIME_DRIVE_AGE * age_scaled) * dur_driving


# Associate utility functions with the mode choice
V = {
    1: V_WALK,    # Walking
    2: V_CYCLE,   # Cycling
    3: V_PT,      # Public Transport
    4: V_DRIVE    # Driving
}

# Specify the model
model_3_power = loglogit({1: V_WALK, 2: V_CYCLE, 3: V_PT, 4: V_DRIVE}, availability, travel_mode)

### Results

In [11]:
biogeme_power = bio.BIOGEME(database, model_3_power)
biogeme_power.modelName = 'model_3_power'

results_m3_power = biogeme_power.estimate()

print("Estimation results for Model 3 Power:")
print(results_m3_power.get_estimated_parameters())

Estimation results for Model 3 Power:
                     Value  Rob. Std err  Rob. t-test  Rob. p-value
ASC_CYCLE        -4.624937      0.201920   -22.904781  0.000000e+00
ASC_DRIVE        -2.137029      0.149775   -14.268303  0.000000e+00
ASC_PT           -2.535221      0.155914   -16.260400  0.000000e+00
B_COST           -0.302618      0.069201    -4.373060  1.225171e-05
B_TIME_CYCLE     -5.608778      0.835329    -6.714452  1.887734e-11
B_TIME_CYCLE_AGE -3.348358      1.705597    -1.963159  4.962771e-02
B_TIME_DRIVE     -6.936331      0.537057   -12.915440  0.000000e+00
B_TIME_DRIVE_AGE -0.168530      1.784207    -0.094456  9.247466e-01
B_TIME_PT        -2.810158      0.430754    -6.523807  6.854473e-11
B_TIME_PT_AGE    -2.155117      0.896292    -2.404481  1.619544e-02
B_TIME_WALK      -7.825469      0.491123   -15.933820  0.000000e+00
B_TIME_WALK_AGE  -3.616616      0.977069    -3.701494  2.143340e-04
cube_cost_coef   -0.000145      0.000289    -0.502135  6.155728e-01
square_cos

In [12]:
print(results_m3_power.print_general_statistics())

Number of estimated parameters:	14
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-4391.936
Final log likelihood:	-4185.768
Likelihood ratio test for the init. model:	412.3361
Rho-square for the init. model:	0.0469
Rho-square-bar for the init. model:	0.0438
Akaike Information Criterion:	8399.536
Bayesian Information Criterion:	8490.777
Final gradient norm:	6.0180E-01
Nbr of threads:	16



### Test Against Model 2

In [13]:
folder_path = os.path.join(os.pardir, 'Model_2')
file_path = os.path.join(folder_path, 'Model_2B.pickle')

# open a file, where you stored the pickled data
file = open(file_path, 'rb')

# dump information to that file
results_m2 = pickle.load(file)

# close the file
file.close()

In [14]:
loglikehood_m3_power = results_m3_power.data.logLike
num_params_m3_power = results_m3_power.data.nparam

loglikehood_m2 = results_m2.logLike
num_params_m2 = results_m2.nparam

# Calculate the LR statistic
LR = 2 * (loglikehood_m3_power - loglikehood_m2)

# Degrees of freedom
df = num_params_m3_power - num_params_m2

# Critical value at 0.05 significance level
critical_value = chi2.ppf(0.95, df)

print("Likelihood Ratio:", LR)
print("Degrees of Freedom:", df)
print("Critical Chi-Square Value (0.05 significance):", critical_value)

if LR > critical_value:
    print("Model 3 Power is significantly better than Model 2.")
else:
    print("No significant improvement in Model 3 Power over Model 2.")

Likelihood Ratio: 15.97390018306578
Degrees of Freedom: 2
Critical Chi-Square Value (0.05 significance): 5.991464547107979
Model 3 Power is significantly better than Model 2.


## Piecewise Linear

In [15]:
thresholds = [0, 1, 5, None]

beta_cost_0_5 = Beta('beta_cost_0_5', 0, None, None, 0)
beta_cost_5_10 = Beta('beta_cost_5_10', 0, None, None, 0)
beta_cost_10_more = Beta('beta_cost_10_more', 0, None, None, 0)
betas_piecewise = [beta_cost_0_5, 
                   beta_cost_5_10, 
                   beta_cost_10_more]

piecewise_cost_driving = piecewise_formula(
    'cost_driving', thresholds, betas_piecewise
)

piecewise_cost_pt = piecewise_formula(
    'cost_transit', thresholds, betas_piecewise
)


In [16]:
# Utility functions with interaction terms
V_WALK = (B_TIME_WALK + B_TIME_WALK_AGE * age_scaled) * dur_walking
V_CYCLE = ASC_CYCLE + (B_TIME_CYCLE + B_TIME_CYCLE_AGE* age_scaled) * dur_cycling
V_PT = ASC_PT + B_COST * piecewise_cost_pt + (B_TIME_PT + B_TIME_PT_AGE* age_scaled) * dur_pt
V_DRIVE = ASC_DRIVE + B_COST * piecewise_cost_driving + (B_TIME_DRIVE + B_TIME_DRIVE_AGE * age_scaled) * dur_driving

# Associate utility functions with the mode choice
V = {
    1: V_WALK,    # Walking
    2: V_CYCLE,   # Cycling
    3: V_PT,      # Public Transport
    4: V_DRIVE    # Driving
}

# Specify the model
model_3_piecewise = loglogit({1: V_WALK, 2: V_CYCLE, 3: V_PT, 4: V_DRIVE}, availability, travel_mode)

### Results

In [17]:
biogeme_piecewise = bio.BIOGEME(database, model_3_piecewise)
biogeme_piecewise.modelName = 'model_3_piecewise'

results_m3_piecewise = biogeme_piecewise.estimate()

print("Estimation results for Model 3 piecewise:")
print(results_m3_piecewise.get_estimated_parameters())

Estimation results for Model 3 piecewise:
                      Value  Rob. Std err  Rob. t-test  Rob. p-value
ASC_CYCLE         -4.720307      0.208802   -22.606666  0.000000e+00
ASC_DRIVE         -2.100906      0.149973   -14.008587  0.000000e+00
ASC_PT            -2.431749      0.154317   -15.758114  0.000000e+00
B_COST            -0.750740      0.092665    -8.101647  4.440892e-16
B_TIME_CYCLE      -5.544459      0.840443    -6.597065  4.193756e-11
B_TIME_CYCLE_AGE  -3.229092      1.694606    -1.905512  5.671351e-02
B_TIME_DRIVE      -6.562500      0.503915   -13.023029  0.000000e+00
B_TIME_DRIVE_AGE   0.064815      1.731553     0.037432  9.701406e-01
B_TIME_PT         -2.980603      0.424759    -7.017167  2.264189e-12
B_TIME_PT_AGE     -2.161462      0.884340    -2.444152  1.451932e-02
B_TIME_WALK       -7.974079      0.495899   -16.080041  0.000000e+00
B_TIME_WALK_AGE   -3.594812      0.985695    -3.646981  2.653400e-04
beta_cost_0_5      0.929906      0.077124    12.057250  0.000

In [18]:
print(results_m3_piecewise.print_general_statistics())

Number of estimated parameters:	15
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-4302.338
Final log likelihood:	-4174.126
Likelihood ratio test for the init. model:	256.423
Rho-square for the init. model:	0.0298
Rho-square-bar for the init. model:	0.0263
Akaike Information Criterion:	8378.253
Bayesian Information Criterion:	8476.011
Final gradient norm:	1.3749E-01
Nbr of threads:	16



### Test Against Model 2

In [19]:
folder_path = os.path.join(os.pardir, 'Model_2')
file_path = os.path.join(folder_path, 'Model_2B.pickle')

# open a file, where you stored the pickled data
file = open(file_path, 'rb')

# dump information to that file
results_m2 = pickle.load(file)

# close the file
file.close()

In [20]:
loglikehood_m3_piecewise = results_m3_piecewise.data.logLike
num_params_m3_piecewise = results_m3_piecewise.data.nparam

loglikehood_m2 = results_m2.logLike
num_params_m2 = results_m2.nparam

# Calculate the LR statistic
LR = 2 * (loglikehood_m3_piecewise - loglikehood_m2)

# Degrees of freedom
df = num_params_m3_piecewise - num_params_m2

# Critical value at 0.05 significance level
critical_value = chi2.ppf(0.95, df)

print("Likelihood Ratio:", LR)
print("Degrees of Freedom:", df)
print("Critical Chi-Square Value (0.05 significance):", critical_value)

if LR > critical_value:
    print("Model 3 piecewise is significantly better than Model 2.")
else:
    print("No significant improvement in Model 3 piecewise over Model 2.")

Likelihood Ratio: 39.257451634570316
Degrees of Freedom: 3
Critical Chi-Square Value (0.05 significance): 7.814727903251179
Model 3 piecewise is significantly better than Model 2.
